### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR(pair-wise loss) на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP(list-wise loss) на implicit данных

Мягкий дедлайн 28 Сентября (пишутся замечания, выставляется оценка, есть возможность исправить до жесткого дедлайна)

Жесткий дедлайн 5 Октября (Итоговая проверка)

In [1]:
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp

from scipy.sparse import random
from scipy import stats
from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error
import scipy.sparse

from scipy.sparse.linalg import spsolve

from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import trange

В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [2]:
ratings = pd.read_csv('ml-1m/ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [3]:
movie_info = pd.read_csv('ml-1m/movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python')

Explicit данные

In [4]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [5]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [6]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [7]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [ ]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [ ]:
model.fit(user_item_t_csr)

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [ ]:
movie_info.head(5)

In [116]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [ ]:
get_similars(1, model)

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [ ]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [ ]:
get_user_history(4, implicit_ratings)

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [118]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [ ]:
get_recommendations(4, model)

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

In [141]:
class MF:
    def recommend(self, user_id, user_item_csr, n=10): 
        user_emb = self.user_factors[user_id]
        predicted_rating = user_emb.dot(self.item_factors.T)
        user_hist = user_item_csr[user_id]
        hist_size = user_hist.count_nonzero()
        top_preds = np.argsort(predicted_rating)[-(n + hist_size + 1):]
        new_recs = np.setdiff1d(top_preds, user_hist.nonzero()[1])
        return np.flip(new_recs[-n:]).reshape(-1, 1)

### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

In [142]:
users = ratings["user_id"]
movies = ratings["movie_id"]
user_item = sp.coo_matrix((ratings["rating"], (users, movies)))

In [143]:
class SGD(MF):
    def __init__(self, lr=0.02, lamb=1e-2, feat_num=76, num_epochs=1):
        self.lr = lr
        self.lamb = lamb
        self.feat_num = feat_num
        self.num_epochs = num_epochs
        
    def fit(self, user_item, ratings):
        self.user_factors = np.random.uniform(0, 1/np.sqrt(self.feat_num), (user_item.shape[0], self.feat_num))
        self.item_factors = np.random.uniform(0, 1/np.sqrt(self.feat_num), (self.feat_num, user_item.shape[1]))
        
        B_w = np.random.normal(0, 1, (user_item.shape[0]))
        B_h = np.random.normal(0, 1, (user_item.shape[1]))

        gen_bias = np.mean(ratings)
        for _ in tqdm(range(self.num_epochs)):
            for i, j, v in zip(tqdm(user_item.row, leave=False,), user_item.col, user_item.data):
                error = (self.user_factors[i, :] @ self.item_factors[:, j] + B_w[i] +  B_h[j] + gen_bias) - v
                W_i_old = self.user_factors[i, :].copy()
                B_w[i] -= self.lr * (error + self.lamb * B_w[i])
                B_h[j] -= self.lr * (error + self.lamb * B_h[j])
                self.user_factors[i, :] -= self.lr * (error * self.item_factors[:, j].T + self.lamb * (self.user_factors[i, :] + B_w[i]))
                self.item_factors[:, j] -= self.lr * (error * W_i_old.T + self.lamb * (self.item_factors[:, j] + B_h[j]))
            print(mean_squared_error((self.user_factors @ self.item_factors + B_w.reshape(-1, 1) + B_h.reshape(1, -1) + gen_bias)[user_item.row, user_item.col], user_item.data))
        self.item_factors = self.item_factors.T
        return self
    

In [144]:
sg = SGD()

In [145]:
sg = sg.fit(user_item, ratings['rating'].to_numpy())

0.9075691424076567



In [113]:
get_similars_items = lambda item_id, sim : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                        for x in reversed(np.argsort(sim[item_id,:])[-10:])]

get_similars_items(1, cosine_similarity(sg.item_factors.T))

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 '1180    Raiders of the Lost Ark (1981)',
 '1179    Princess Bride, The (1987)',
 '1132    Wrong Trousers, The (1993)',
 '735    Close Shave, A (1995)',
 '148    Apollo 13 (1995)',
 '315    Shawshank Redemption, The (1994)',
 '1222    Glory (1989)',
 '1242    Great Escape, The (1963)']

In [114]:

get_similars_items(1982, cosine_similarity(H.T))

['1913    Halloween (1978)',
 '1324    Carrie (1976)',
 '1326    Nightmare on Elm Street, A (1984)',
 '1114    Howling, The (1980)',
 '2292    Pink Flamingos (1972)',
 '2593    War of the Worlds, The (1953)',
 '956    Night of the Living Dead (1968)',
 '1329    Omen, The (1976)',
 '2390    Texas Chainsaw Massacre, The (1974)',
 '1925    Poltergeist (1982)']

In [115]:

get_similars_items(2628, cosine_similarity(H.T))

['2559    Star Wars: Episode I - The Phantom Menace (1999)',
 '3249    Deterrence (1998)',
 '877    1-900 (1994)',
 '3159    Wirey Spindell (1999)',
 '1192    Star Wars: Episode VI - Return of the Jedi (1983)',
 '675    Tigrero: A Film That Was Never Made (1994)',
 '2468    Beyond the Poseidon Adventure (1979)',
 '345    Clear and Present Danger (1994)',
 '2412    My Name Is Joe (1998)',
 '2182    Cabinet of Dr. Ramirez, The (1991)']

In [146]:
get_recommendations(4, sg)

['3724    X-Men (2000)',
 '3406    Place in the Sun, A (1951)',
 '3013    World Is Not Enough, The (1999)',
 '2970    Trading Places (1983)',
 '2964    Spaceballs (1987)',
 '2951    Falling Down (1993)',
 '2020    Rescuers Down Under, The (1990)',
 '2019    Popeye (1980)',
 '2012    Little Mermaid, The (1989)',
 '2011    Lady and the Tramp (1955)']

Так и не получилось подобрать классные параметры для SGD :(
Но все равно вроде рекоммендации +- адекватны

### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

In [ ]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()


In [ ]:
class ALS:
    def __init__(self, lamb, n_iters, n_factors, alpha):
        self.lamb = lamb
        self.alpha = alpha
        self.n_iters = n_iters
        self.n_factors = n_factors
    
    def fit(self, ratings):
        #У нас спарс матрицы - единички просто так не прибавить, попробуем это сделать в другом месте
        Cui = ratings.copy().tocsr()
        Cui.data *= self.alpha
        Ciu = Cui.T.tocsr()
        self.n_users, self.n_items = Cui.shape

        rstate = np.random.RandomState(228)
        self.W = scipy.sparse.csr_matrix(rstate.normal(size = (self.n_users, self.n_factors)))
        self.H = scipy.sparse.csr_matrix(rstate.normal(size = (self.n_items, self.n_factors)))
        
        for _ in trange(self.n_iters, desc = 'training'):
            self._als_step(Cui, self.W, self.H, self.n_users)
            self._als_step(Ciu, self.H, self.W, self.n_items)
        
        return self
    
    def _als_step(self, Cui, X, Y, n_shag):
        YtY = Y.T.dot(Y)
        Y_eye = scipy.sparse.eye(Y.shape[0])
        lambda_eye = self.lamb * scipy.sparse.eye(self.n_factors)
        for u in tqdm(range(n_shag)):
            conf_samp = Cui[u,:].toarray()
            pref = conf_samp.copy()
            pref[pref != 0] = 1
            cui_loc = scipy.sparse.diags(conf_samp, [0])
            A = YtY + Y.T.dot(cui_loc).dot(Y)+lambda_eye
            b = Y.T.dot(cui_loc+Y_eye).dot(pref.T)
            
            X[u] = spsolve(A, b)

        return self
als = ALS(n_iters = 15, n_factors = 64, alpha = 15, lamb = 0.01)
als.fit(user_item_csr)

In [ ]:
get_similars = lambda item_id, sim : [movie_info[movie_info["movie_id"] == x]["name"].to_string()
                                        for x in reversed(np.argsort(sim[item_id,:])[-10:])]

get_similars(1, cosine_similarity(als.H))

In [ ]:

get_similars = lambda item_id, sim : [movie_info[movie_info["movie_id"] == x]["name"].to_string()
                                        for x in reversed(np.argsort(sim[item_id,:])[-10:])]

get_similars(2628, cosine_similarity(als.H))

Если что, у меня уже здесь были норм рекоммендации, вы мне поставили 25, можете посмотреть прошлый коммит, я просто случайно затер, а переобучить уже не успевал

### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных


In [154]:
class BPR(MF):
    def __init__(self, n_factors=100, lr=0.1, lamb=0.1):
        self.n_factors = n_factors
        self.lr = lr
        self.lamb = lamb
        
    
    def fit(self, user_item_csr, n_epochs=15):
        self.n_users, self.n_items = user_item_csr.shape
        self.user_factors = np.random.normal(0., 1./np.sqrt(self.n_factors),
                                             size=(self.n_users, self.n_factors))
        self.item_factors = np.random.normal(0., 1./np.sqrt(self.n_factors),
                                             size=(self.n_items, self.n_factors))
        
        self.zero_rows = (user_item_csr.getnnz(1) == 0).nonzero()[0]
        for epoch in trange(n_epochs):
            rows, cols = user_item_csr.nonzero()
            loss = 0.
            for user_id, pos_item_id in zip(tqdm(rows), cols):
                neg_item_id = np.random.randint(1, self.n_items)
                while user_item_csr[user_id, neg_item_id] > 0:
                    neg_item_id = np.random.randint(1, self.n_items)
                
                user_u = self.user_factors[user_id]
                item_i = self.item_factors[pos_item_id]
                item_j = self.item_factors[neg_item_id]
                
                r_ui = user_u.dot(item_i.T)
                r_uj = user_u.dot(item_j.T)
                r_uij = r_ui - r_uj

                sigmoid = np.exp(-r_uij) / (1. + np.exp(-r_uij))
            
                self.user_factors[user_id] -= self.lr * (sigmoid * (self.item_factors[neg_item_id] - self.item_factors[pos_item_id]) + self.lamb * self.user_factors[user_id])
                self.item_factors[pos_item_id] -= self.lr * (sigmoid * (-self.user_factors[user_id]) + self.lamb * self.item_factors[pos_item_id])
                self.item_factors[neg_item_id] -= self.lr * (sigmoid * self.user_factors[user_id] + self.lamb * self.item_factors[neg_item_id])
                loss += np.log(sigmoid)

            
            loss /= len(rows)
            print(f'Cur loss: {loss}')

In [156]:
bpr_model = BPR(n_factors=64, lr=0.07, lamb=0.000001)
bpr_model.fit(user_item_csr, n_epochs=6)

  0%|          | 0/6 [00:00<?, ?it/s]

 17%|█▋        | 1/6 [00:37<03:09, 37.98s/it]


Cur loss: -1.7650790420643565


 33%|███▎      | 2/6 [01:17<02:34, 38.57s/it]


Cur loss: -3.4272270219319645


...


Cur loss: -4.907923534845171


 83%|████████▎ | 5/6 [03:16<00:39, 39.18s/it]


Cur loss: -5.520148785874685


100%|██████████| 6/6 [03:54<00:00, 39.16s/it]


Cur loss: -6.2325779187190715


In [75]:
get_similars = lambda item_id, sim : [movie_info[movie_info["movie_id"] == x]["name"].to_string()
                                        for x in reversed(np.argsort(sim[item_id,:])[-10:])]

get_similars(1, cosine_similarity(bpr_model.item_factors))

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '3682    Chicken Run (2000)',
 '2252    Pleasantville (1998)',
 '584    Aladdin (1992)',
 '1250    Back to the Future (1985)',
 '1838    Mulan (1998)',
 '360    Lion King, The (1994)',
 '591    Beauty and the Beast (1991)']

In [158]:
get_recommendations(4, bpr_model)

['3509    Gladiator (2000)',
 '3352    Animal House (1978)',
 '2879    From Russia with Love (1963)',
 '1931    Lethal Weapon (1987)',
 '1353    Star Trek: The Wrath of Khan (1982)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '1239    Stand by Me (1986)',
 '1230    Bridge on the River Kwai, The (1957)',
 '1203    Godfather: Part II, The (1974)',
 '1192    Star Wars: Episode VI - Return of the Jedi (1983)']

Можно поподбирать параметры и немного улучшить результаты, естественно

### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных

In [147]:
class WARP(MF):
    def __init__(self, n_factors=100, lr=0.1, lamb=0.1):
        self.n_factors = n_factors
        self.lr = lr
        self.lamb = lamb
        self.M = 1
        self.max_negatives = 10
    
    
    def fit(self, user_item_csr, n_epochs=15):
        self.n_users, self.n_items = user_item_csr.shape
        self.user_factors = np.random.uniform(0., 1./np.sqrt(self.n_factors),
                                             size=(self.n_users, self.n_factors))
        self.item_factors = np.random.uniform(0., 1./np.sqrt(self.n_factors),
                                             size=(self.n_items, self.n_factors))
        
        self.zero_rows = (user_item_csr.getnnz(1) == 0).nonzero()[0]
        for epoch in trange(n_epochs):
            rows, cols = user_item_csr.nonzero()
            sum_loss = 0.
            for user_id, pos_item_id in zip(tqdm(rows), cols):
                score_pos = self.user_factors[user_id].dot(self.item_factors[pos_item_id])
                for n in range(1, self.max_negatives + 1):
                    neg_item_id = np.random.randint(1, self.n_items)
                    while user_item_csr[user_id, neg_item_id] > 0:
                        neg_item_id = np.random.randint(1, self.n_items)
                        
                    score_neg = self.user_factors[user_id].dot(self.item_factors[neg_item_id])
                    if self.M + score_neg - score_pos > 0:
                        rank_approx = self.max_negatives / n
                        rank_loss = np.log(rank_approx)
                        loss = rank_loss #(self.M + score_neg - score_pos)

                        self.user_factors[user_id] -= self.lr * (rank_loss * (self.item_factors[neg_item_id] - self.item_factors[pos_item_id]) + self.lamb * self.user_factors[user_id])
                        self.item_factors[pos_item_id] -= self.lr * (rank_loss * (-self.user_factors[user_id]) + self.lamb * self.item_factors[pos_item_id])
                        self.item_factors[neg_item_id] -= self.lr * (rank_loss * self.user_factors[user_id] + self.lamb * self.item_factors[neg_item_id])
                        sum_loss += loss
                        
                        break

            
            sum_loss /= len(rows)
            print(f'Cur loss: {sum_loss}')

In [148]:
warp_model = WARP(n_factors=64, lr=0.02, lamb=0.0000001)

warp_model.fit(user_item_csr, n_epochs=6)

In [150]:
get_similars = lambda item_id, sim : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                        for x in reversed(np.argsort(sim[item_id,:])[-10:])]

get_similars(2628, cosine_similarity(warp_trained.item_factors))

['2559    Star Wars: Episode I - The Phantom Menace (1999)',
 '2502    Matrix, The (1999)',
 '1192    Star Wars: Episode VI - Return of the Jedi (1983)',
 '476    Jurassic Park (1993)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '1335    Star Trek: First Contact (1996)',
 '1351    Star Trek VI: The Undiscovered Country (1991)',
 '108    Braveheart (1995)',
 '1851    Small Soldiers (1998)',
 '1505    Lost World: Jurassic Park, The (1997)']

In [153]:
get_similars(1, cosine_similarity(warp_trained.item_factors))

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 '2618    Tarzan (1999)',
 "2286    Bug's Life, A (1998)",
 '584    Aladdin (1992)',
 '591    Beauty and the Beast (1991)',
 '1526    Hercules (1997)',
 '12    Balto (1995)',
 '33    Babe (1995)',
 '592    Pinocchio (1940)']

In [151]:
get_recommendations(4, warp_trained)

['3402    Close Encounters of the Third Kind (1977)',
 '2875    Dirty Dozen, The (1967)',
 '1568    Hunt for Red October, The (1990)',
 '1230    Bridge on the River Kwai, The (1957)',
 '1214    Boat, The (Das Boot) (1981)',
 '1204    Full Metal Jacket (1987)',
 '1203    Godfather: Part II, The (1974)',
 '1190    Apocalypse Now (1979)',
 '1182    Aliens (1986)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...']